## study multilabel stratification function 

study https://stackoverflow.com/questions/53378970/how-to-perform-multilabel-stratified-sampling

In [1]:
# example of a multi-label classification task
from sklearn.datasets import make_multilabel_classification
import numpy as np 
# define dataset
X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)
# summarize dataset shape
print(X.shape, y.shape)
# summarize first few examples
for i in range(10):
	print(X[i], y[i])

(1000, 10) (1000, 3)
[ 3.  3.  6.  7.  8.  2. 11. 11.  1.  3.] [1 1 0]
[7. 6. 4. 4. 6. 8. 3. 4. 6. 4.] [0 0 0]
[ 5.  5. 13.  7.  6.  3.  6. 11.  4.  2.] [1 1 0]
[1. 1. 5. 5. 7. 3. 4. 6. 4. 4.] [1 1 1]
[ 4.  2.  3. 13.  7.  2.  4. 12.  1.  7.] [0 1 0]
[ 4.  3.  3.  2.  5.  2.  3.  7.  2. 10.] [0 0 0]
[ 3.  3.  3. 11.  6.  3.  4. 14.  1.  3.] [0 1 0]
[ 2.  1.  7.  8.  4.  5. 10.  4.  6.  6.] [1 1 1]
[ 5.  1.  9.  5.  3.  4. 11.  8.  1.  8.] [1 1 1]
[ 2. 11.  7.  6.  2.  2.  9. 11.  9.  3.] [1 1 1]


In [2]:
from skmultilearn.model_selection import iterative_train_test_split
x_train, y_train, x_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [10]:
#def calc_prop(arr):
#    return arr.sum(axis=0)/arr.shape[0]
#
#def calc_prop_change(arr, base_arr):
#    thing = (calc_prop(arr)-calc_prop(base_arr))/calc_prop(base_arr)
#    #return np.array([round(thing_i,5) for thing_i in thing])
#    return thing


def calc_prop(arr, set_thing = None ):
    #set_thing : optional input 
    aggregate = []

    if set_thing == None  : 
        if type(arr) == np.ndarray : 
            set_thing = set(arr.flatten())
        else : 
            set_thing = set(arr)
    else : 
        set_thing = set_thing
    for i in set_thing:
        aggregate.append(np.count_nonzero(arr == i, axis=0))
    return np.array(aggregate)/arr.shape[0]


def calc_prop_change(arr, base_arr, set_thing = None):
    thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
    return thing

In [11]:
##base proportions
100*calc_prop(y)

array([[36.3, 26.9, 79.5],
       [63.7, 73.1, 20.5]])

In [12]:
##y-wise difference
100*calc_prop_change(y_train,y), 100*calc_prop_change(y_test,y) #percentage wise

(array([[-0.13774105, -0.0929368 ,  1.41509434],
        [ 0.07849294,  0.03419973, -5.48780488]]),
 array([[ 0.55096419,  0.37174721, -5.66037736],
        [-0.31397174, -0.13679891, 21.95121951]]))

In [13]:
##can see that division was well done, and the proportions was somewhat well preserved for all the three categories
#however, NOT cross validation (only one split)

# Cross validaiton using multilabel 
* http://scikit.ml/concepts.html#The-multi-label-data-representation, http://scikit.ml/stratification.html

look at this! (iterative stratification) : http://scikit.ml/api/skmultilearn.model_selection.iterative_stratification.html#module-skmultilearn.model_selection.iterative_stratification => this seems to be the kfold versio nthingie


* X : (n_samples, n_features) (dense ones like np arrays are possible)
* y : (n_samples, n_labels)
    * **the labels in y shouldn't be mmore than binary class!** (see below)
> y is expected to be a binary integer indicator matrix of shape. In the binary indicator matrix each matrix element A[i,j] should be either 1 if label j is assigned to an object no i, and 0 if not.
    * therefore, stratified 할때 binary 해야함!
        * **sklearn은 binary/multiclass 다되는데.... 이거는 안되나?**

## 2.1. ABCD config에서 가져오기 


In [14]:
import sys 
import pandas as pd 
import numpy as np 
label_csv = '/scratch/connectome/dyhan316/VAE_ADHD/junbeom_finetuning/csv/ABCD_csv/ABCD_phenotype_total_ONLY_MRI.csv'

In [56]:
from sklearn.model_selection import StratifiedKFold, KFold
kf = StratifiedKFold(n_splits=5)

labels = pd.read_csv(label_csv)

label_name = "sex"
labels = labels[labels[label_name].notna()] #이파트를 더 추가해야할듯 (if doing stuff)
labels = labels[labels['Panic.Disorder.x'].notna()] #이파트를 더 추가해야할듯 (if doing stuff)
labels[label_name] = labels[label_name].astype('str') #regressino하려면 이파트를 바꿔야할듯 

task_include = ['1.0','2.0']
data_1 = labels[labels[label_name] == task_include[0]]
data_2 = labels[labels[label_name] == task_include[1]]
data_1 = data_1[:50]
data_2 = data_2[:50]

label_tv = pd.concat([data_1, data_2])

col2view = ['sex','Panic.Disorder.x']

##default thing 


In [57]:
[i for i in label_tv.keys()]


['Unnamed: 0',
 'SubjectID',
 'age',
 'sex',
 'race.ethnicity',
 'high.educ',
 'income',
 'married',
 'abcd_site',
 'height',
 'weight',
 'BMI',
 'vol',
 'nihtbx_totalcomp_uncorrected',
 'nihtbx_fluidcomp_uncorrected',
 'nihtbx_pattern_uncorrected',
 'nihtbx_picture_uncorrected',
 'nihtbx_list_uncorrected',
 'nihtbx_flanker_uncorrected',
 'nihtbx_cardsort_uncorrected',
 'nihtbx_cryst_uncorrected',
 'nihtbx_reading_uncorrected',
 'nihtbx_picvocab_uncorrected',
 'poa_total_6month_follow_up',
 'friends',
 'close_friends',
 'upps_negative_urgency',
 'upps_lack_of_planning',
 'upps_sensation_seeking',
 'upps_positive_urgency',
 'upps_lack_of_perseverance',
 'upps_total',
 'AnxDep.CBCL',
 'WithDep.CBCL',
 'Somatic.CBCL',
 'Social.CBCL',
 'Thought.CBCL',
 'Attention.CBCL',
 'RuleBreak.CBCL',
 'Aggressive.CBCL',
 'Internal.CBCL',
 'External.CBCL',
 'TotProb.CBCL',
 'Depress.CBCL',
 'AnxDisord.CBCL',
 'SomaticPr.CBCL',
 'ADHD.CBCL',
 'Opposit.CBCL',
 'Conduct.CBCL',
 'Sluggish.Cognitive.Tempo..

In [58]:

##first, get the things

#print(calc_prop(label_tv[['sex','married']].values))
print(col2view)
print(calc_prop(label_tv[col2view].values))
set_thing = set(label_tv[col2view].values.flatten()) #un split된 상태에서의 set을 써야함



for FOLD, (train_idx, valid_idx) in enumerate(kf.split(label_tv, label_tv[label_name])): 
    print(f"===FOLD : {FOLD}===")
    train = label_tv.iloc[train_idx]
    valid = label_tv.iloc[valid_idx]

    print("with training")
    print(calc_prop_change(train[col2view].values, label_tv[col2view].values, set_thing = set_thing))
    
    print("with validation")
    print(calc_prop_change(valid[col2view].values, label_tv[col2view].values, set_thing = set_thing))

['sex', 'Panic.Disorder.x']
[[0.   0.99]
 [0.5  0.  ]
 [0.5  0.  ]
 [0.   0.01]]
===FOLD : 0===
with training
[[        nan  0.01010101]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan -1.        ]]
with validation
[[        nan -0.04040404]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan  4.        ]]
===FOLD : 1===
with training
[[        nan -0.00252525]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan  0.25      ]]
with validation
[[        nan  0.01010101]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan -1.        ]]
===FOLD : 2===
with training
[[        nan -0.00252525]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan  0.25      ]]
with validation
[[        nan  0.01010101]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan -1.        ]]
===FOLD : 3===
with training
[[        nan -0.00252525]
 [ 0.                 nan]
 [ 0.                 nan]
 [        nan  0.25 

/tmp/ipykernel_1471257/4080913144.py:27: RuntimeWarning: invalid value encountered in divide
  thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
/tmp/ipykernel_1471257/4080913144.py:27: RuntimeWarning: invalid value encountered in divide
  thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
/tmp/ipykernel_1471257/4080913144.py:27: RuntimeWarning: invalid value encountered in divide
  thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
/tmp/ipykernel_1471257/4080913144.py:27: RuntimeWarning: invalid value encountered in divide
  thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
/tmp/ipykernel_1471257/4080913144.py:27: RuntimeWarning: invalid value encountered in divide
  thing = (calc_prop(arr, set_thing)-calc_prop(base_arr,set_thing))/calc_prop(base_arr, set_thing)
/tmp/ipykernel_1471257/4080913144.py:27:

In [59]:
train[['sex','married']].values


array([['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 4.0],
       ['1.0', 1.0],
       ['1.0', nan],
       ['1.0', 1.0],
       ['1.0', 5.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 6.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 4.0],
       ['1.0', 5.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 3.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 3.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 5.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 6.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 1.0],
       ['1.0', 4.0],
       ['1.0', 1.0],
       ['2.0', 1.0],
       ['2.0', 1.0],
       ['2.0', 1.0],
       ['2.0', 6.0],
       ['2.0', 5.0],
       ['2.0', 3.0],
       ['2.0', 1.0],
       ['2.0'

In [60]:
label_tv.iloc[train_idx]

,Unnamed: 0,SubjectID,age,sex,race.ethnicity,high.educ,income,married,abcd_site,height,...,Substance.Use.Disorder.x,Other.Drugs.Use.Disorder.x,Sedative.Hypnotic.or.Anxiolytic.Use.Disorder.x,Other.Specified.Trauma.and.Stressor.Related.Disorder.x,Post.Traumatic.Stress.Disorder.x,HOMICIDAL.IDEATION.AND.BEHAVIOR.x,totalBIS,totalBAS,PPSP_frequency,PPSP_distress
1,1,NDARINV007W6H7B,126.0,1.0,1.0,19.0,10.0,1.0,22.0,56.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,7,18
2,2,NDARINV00BD7VDC,112.0,1.0,1.0,20.0,10.0,1.0,7.0,57.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,13.0,0,0
3,3,NDARINV00CY2MDM,130.0,1.0,1.0,15.0,6.0,4.0,20.0,56.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,10.0,0,0
4,4,NDARINV00HEV6HB,124.0,1.0,2.0,13.0,NaN,1.0,12.0,57.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,24.0,1,0
5,5,NDARINV00J52GPG,110.0,1.0,5.0,21.0,6.0,NaN,17.0,50.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,24.0,8,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,70,NDARINV06BKAHN5,109.0,2.0,2.0,17.0,7.0,1.0,14.0,53.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,33.0,5,14
81,81,NDARINV07CMT243,123.0,2.0,3.0,8.0,1.0,4.0,10.0,56.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,25.0,0,0
82,82,NDARINV07HGA3B7,114.0,2.0,1.0,18.0,7.0,1.0,16.0,55.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,25.0,0,0
84,84,NDARINV07RAHHYH,119.0,2.0,4.0,19.0,10.0,1.0,8.0,53.140000,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,15.0,0,0


In [ ]:
train_idx

In [ ]:
label_tv[label_name]

In [ ]:
np.max(train[label_name])

In [ ]:
set(train[label_name])

In [ ]:
aggregate = []
arr = train[label_name]
for i in set(train[label_name]):
    aggregate.append(np.count_nonzero(arr == i, axis=0))
print(np.array(aggregate)/arr.shape[0])

## 2.2. now doing iterative stratification
(http://scikit.ml/api/skmultilearn.model_selection.iterative_stratification.html#module-skmultilearn.model_selection.iterative_stratification) 



In [ ]:
from skmultilearn.model_selection import IterativeStratification
k_fold = IterativeStratification(n_splits=5, order=1) #on't increae the order (i think)(if > 1, does with replacement)
print(100*calc_prop(y))

total_val_set = set()

for train,test in k_fold.split(X,y):
    x_train,y_train = X[train], y[train]
    x_test,y_test = X[test], y[test]
    print("\n===y_train, y_test percentage deviations from y (%)===")
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    
    print(100*calc_prop_change(y_train,y))
    print(100*calc_prop_change(y_test,y))
    #print(calc_prop_change(train,X))
    
    total_val_set = total_val_set.union(train)


In [ ]:
len(total_val_set) #as expected => i.e. didn't reuse the things, but CV was actually well done 

## NOW HAVE TO CHECK IF multiclass + multilabel works!(i.e. try multiclass)

In [ ]:
#y_multiclass (modifing)
y_multiclass = y.copy()
y_multiclass[::10,0] = 2 #changing the first labels to 2 (multiclass) every 5 things
#y_multiclass[::11,0] = 3 #changing the first labels to 2 (multiclass) every 5 things

In [ ]:
y_multiclass

In [ ]:
from skmultilearn.model_selection import IterativeStratification
k_fold = IterativeStratification(n_splits=5, order = 1) #on't increae the order (i think)(if > 1, does with replacement)
print("below : the shapes of (class_i, n_labels)")
print(100*calc_prop(y_multiclass))

total_val_set = set()

for train,test in k_fold.split(X,y_multiclass):
    x_train,y_train = X[train], y_multiclass[train]
    x_test,y_test = X[test], y_multiclass[test]
    #print(x_train.shape, x_test.shape)

    print("\n===y_train, y_test percentage deviations from y (%)===")
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
    print(100*calc_prop_change(y_train,y_multiclass))
    print(100*calc_prop_change(y_test,y_multiclass))
    #print(calc_prop_change(train,X))
    
    total_val_set = total_val_set.union(train)

# 보듯이, https://github.com/scikit-multilearn/scikit-multilearn/issues/132
> multiclass + multilabel은 잘 안되는 것 같다!

https://github.com/trent-b/iterative-stratification/issues/27#issue-1577817136

일단 위에 두개보고, 되는지 안되는지 확인해보자 => 안된다면.. 흠.. 어떻게 하지? 
(그리고 어차피, age같은 것은 하지도 못할거같은데..?)(unless artificially splitting into age brackets and treating age as a categorical thing)

=> 이것 보고 하기!


In [ ]:
aaaa #meant to interrupt it 

In [ ]:
small_y = y_multiclass[:5]
print(np.count_nonzero(small_y == 0, axis = 0),
np.count_nonzero(small_y == 1, axis = 0),
np.count_nonzero(small_y == 2, axis = 0))




In [ ]:
small_y

In [ ]:
100*calc_prop(y)

In [ ]:
np.max(small_y)

In [ ]:
def calc_prop_multiclass(arr):
    aggregate = []
    for i in range(np.max(arr)+1):
        aggregate.append(np.count_nonzero(arr == i, axis=0))
    return np.array(aggregate)/arr.shape[0]



def calc_prop_change_multiclass(arr, base_arr):
    thing = (calc_prop_multiclass(arr)-calc_prop_multiclass(base_arr))/calc_prop_multiclass(base_arr)
    return thing

In [ ]:
calc_prop_multiclass(y_multiclass)

In [ ]:
calc_prop_change_multiclass(y_multiclass[:8],y_multiclass)

In [ ]:
y_multiclass[:10]

In [ ]:
y_multiclass

### past stuff (sparse X, y 쓸대인데... 그냥 일반적인 dense 써도 되어서 아놤)

In [ ]:
from skmultilearn.dataset import load_dataset
X, y, _, _ = load_dataset('emotions', 'train')

In [ ]:
##looking at data
print(X.shape, y.shape)


for i, y_i in enumerate(y):
    print(f"===the {i}th target is===")
    print(y_i)
    if i == 5:
        break
#here, y is sparse (hence where the sparse thing came from)